## CONHEÇA NOSSO BLOG: https://www.dadosedecisao.com/blog

Este notebook foi desenvolvido para trazer em termos práticos o conhecimento sobre regressão logística mostrado no artigo: Regressão Logística: o essencial.
This notebook was developed to show in pratical terms the knowledge about logistic regression at paper: Logistic Regression: the essential.

Importando a base de dados online
Importing the dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
for k_value in range(1,30):
    import numpy as np
    import pandas as pd
    votos = pd.read_csv('/kaggle/input/congressional-voting-records/house-votes-84.csv')

Verificando os 10 primeiros valores da base de dados
The 10 first value

In [ ]:
votos.head(10)

Substituindo os valores y,n,? por 1,0,NaN (missing)
Criando a variável Republican
Substituindo os valores de da variável Class Name (republican e democrat) por (1 e 0);

Changing y,n,? values for 1,0,NaN
Input a new column: Republican
Changing class name values for 1,0

In [ ]:
ynmap = {'y':1,'n':0,'?':np.nan}
partymap = {'republican':0,'democrat':1}
votos['republican'] = votos['Class Name'].map(partymap)
votos.drop('Class Name',axis=1,inplace=True)
for column in votos.columns.drop('republican'):
    votos[column+'1'] = votos[column].map(ynmap)
    votos.drop(column,axis=1,inplace=True)
partymap = {'republican':1,'democrat':0}
data_col = votos.columns

Verificando o tamanho da base de dados (quantidade de linhas totais)
The length of dataset

In [ ]:
len(votos)

# Tratamento de missing

Missing data treatment 

Nesta primeira tentativa, vamos construir um modelo eliminando todos os missing da amostra.

This model will exlude all missing data.

Verificando a quantidade de missing por variável

Missing data by column

In [ ]:
votos.isnull().sum()

Temos um total de 203 linhas sem dados perdidos (47%), o que significa dizer que 53% da nossa amostra possui dados perdidos nas observações. Além disso, 100% das variáveis possui dados perdidos. 

In [ ]:
# identificando o detalhe de cada observação e retornando true para dado perdido e false para dado completo num array
votos.isnull().values.ravel()

In [ ]:
#contagem do total de dados perdidos em toda a tabela (levando em conta observaçõe + variáveis = extensão total dos dados perdidos)
votos.isnull().values.ravel().sum()

# obs: a diferença entre esta linha e a soma, é que a soma retorna o número de dados perdidos por variável

In [ ]:
# eliminando os dados missing
votos = votos.dropna()

In [ ]:
len(votos)

Vemos que tivemos uma redução considerável no número de observações em nossa tabela. A representatividade perdida, após a tratativa de missing excluindo todos os dados perdidos, é de 53%. Para bases de dados com tamanha extensão de dados perdidos, o ideal é realizar uma análise mais profunda acerca dos padrões aleatórios ou não aleatórios dos dados perdidos. Para os intuitos introdutórios desse kernel, vamos optar pela eliminação total dos dados missing.

We using that form to treated missing data because the intention of this notebook is just only show how a logistic regression model works. We know wich the best way to treat missing data is MCAR or CAR analysis when we have a lot of missing values. 

# Análise exploratória de dados

Exploratory analysis

Importando a biblioteca pandas profiling para construir as visualizações gráficas mais comuns em análise exploratória (histogramas, tabelas de correlação, dentre outros). 
A análise exploratória é um dos passos mais importantes de qualquer modelo. Uma análise exploratória mal feita pode enviesar os dados, geralmente, resultando em performance reduzida do modelo.

In [ ]:
import pandas_profiling

In [ ]:
relatorio_votos = pandas_profiling.ProfileReport(votos)

In [ ]:
relatorio_votos.to_file('relatorio_votos.html')

In [ ]:
relatorio_votos

# Divisão em treino e teste

We using train and test division to make our model.

Aqui chegamos no momento de separar nossas bases. O método de divisão de bases em treino e teste é o mais utilizado na comunidade de data science. Geralmente, 70% dos dados vão para a base de treino e 30% ficam na base de teste. O motivo dessa divisão é simples: 1) a base de treino serve para que o modelo execute previsões com os dados brutos. Ele faz isso a partir de tentativa e erro, reduzindo o erro a cada tentativa. No treino ele pode chegar ao overfiting (previsão perfeita) de tanto treinar. Já a 2) base de teste é a mais fundamental, pois nos revelará como o modelo se comporta ao receber novos dados. 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# devinindo x e y
#  x assumira todas as linhas [:] e as colunas de [a:17]
# y assumira a variável republican
X = votos.iloc[:,1:17]
y = votos.republican

# aqui, defino os nomes das bases de treino e teste e o tamanho desta última

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Verificando o tamanho da base de treino e base de teste.
Base de treino tem 162 linhas e 16 colunas
base de teste tem 70 linhas e 16 colunas

In [ ]:
print(X_train.shape)
print(X_test.shape)

# Rodando modelos

Regressão logística

Carregando a biblioteca

In [ ]:
from sklearn.linear_model import LogisticRegression

Configurando a biblioteca

In [ ]:
#criando a variavel votos_logistic que receberá o método de regressão logística
votos_logistic = LogisticRegression(C=10)
#.fit é o que aplica o modelo nas bases de treino inicialmente, geraldo as estatísticas iniciais
votos_logistic.fit(X_train, y_train)
#.predict aplica o aprendido no treino na base de teste
votos_logistic_predictions = votos_logistic.predict(X_test)
# criando a métrica de acurácia
from sklearn.metrics import accuracy_score
acc_votos_logistic = accuracy_score(y_test, votos_logistic_predictions)
print("accuracy_score: %4f" % acc_votos_logistic)

É comum utilizarmos nós "tunarmos" nosso modelo em busca dos melhores parametros. Faço isso abaixo. Esta parte é opcional.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
tuned_parameters_votos_logistic = [{'C': [0.1,1,10]}]
clf_votos_logistic = GridSearchCV(LogisticRegression (), tuned_parameters_votos_logistic, cv=3, scoring='accuracy')
clf_votos_logistic.fit(X_train, y_train)

In [ ]:
print ("Melhores parametros encontrados:")
print()
print(clf_votos_logistic)
print()
print("Grid scores:")
print()
means = clf_votos_logistic.cv_results_['mean_test_score']
stds = clf_votos_logistic.cv_results_['std_test_score']
for mean, std, params in zip (means, stds, clf_votos_logistic.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()
print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
from sklearn.metrics import accuracy_score
acc_votos_logistic = accuracy_score(y_test, votos_logistic_predictions)
print("accuracy_score: %4f" % acc_votos_logistic)


Cálculo do indicador da area under the curve (área sob a curva) que mostra a qualidade da nossa previsão. Tivemos um excelente desempenho, acertando 0,9913. 
Obs: o fato de ter eliminado os missing e reduzido a extensão da base tem impacto direto.

In [ ]:
import sklearn.metrics as metrics

In [ ]:
probs_votos_logistic = votos_logistic.predict_proba(X_test)
from sklearn.metrics import roc_auc_score
auc_votos_logistic = roc_auc_score(y_test, probs_votos_logistic[:,1])
print("AUC: %.4f" % auc_votos_logistic)

Criando a tabela com os detalhes estatísticos (coeficientes, desvio padrão, normalização, p-valor, percentil) e outras métricas como pseudo R, Razão e desigualdades, logit, dentre outros.

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y, X)
result=logit_model.fit(method='bfgs', full_output='bool')
print (result.summary())

# VISUALIZAÇÃO GRÁFICA DA QUALIDADE DO MODELO

Nesta última etapa, construímos a matriz de confusão (gráfico 1), distribuição de probabilidades (gráfico 2) e Curva ROC (gráfico3).
Em resumo, verificamos na matriz de confusão como o modelo performou nas previsões. O verdadeiro negativo e o verdadeiro positivo são os mais importantes, pois representam as previsões corretas.
A distribuição de probabilidades mostra se nossas previsões estão claramente bem distribuídas (no nosso caso, beirou a perfeição). 
E a curva ROC é a representação gráfica da área sob a curva.
Noutro momento, explicarei em detalhes sobre cada um deles em nosso blog.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, confusion_matrix, auc
from sklearn import linear_model

In [ ]:
def evalBinaryClassifierRL (model, x, y_test, labels=['Positives', 'Negatives']):

# model predicts probabilities of positive class

    p= votos_logistic.predict_proba(X_test)
    if len(votos_logistic.classes_)!=2:
        raise ValueError('A binary class problem is required')
    if model.classes_[1] == 1:
        pos_p = p[:,1]
    elif model.classes_[0] == 1:
        pos_p = p[:,0]
        
    #Figure
    plt.figure(figsize=[15,4])
    
    
    # 1 - Confusion matrix
    cm_rl = confusion_matrix(y_test, votos_logistic_predictions)
    plt.subplot(131)
    ax = sns.heatmap(cm_rl, annot=True, cmap='Blues', cbar=False,
                    annot_kws={"size": 14}, fmt='g')
    cmlabels = ['Verdadeiros negativos', 'Falso Positivos', 'Falso Negativos', 'Verdadeiros Positivos']
    
    for i, t in enumerate(ax.texts):
        t.set_text(t.get_text() + "\n" + cmlabels[i])
    plt.title('Matrix de confusão', size =15)
    plt.xlabel('Valores previstos', size = 13)
    plt.ylabel('Valores verdadeiros', size=13)
    
    #2 - Distribuiçãode probabilidades para ambas as classes
    df = pd.DataFrame({'probPos':pos_p, 'target':y_test})
    plt.subplot(132)
    plt.hist(df[df.target==1].probPos, density=True,
            alpha=.5, color='blue', label=labels[0])
    plt.hist(df[df.target==0].probPos, density=True,
            alpha=.5, color='purple', label=labels[1])
    plt.axvline(.5, color='red', linestyle=':', label='Boundary')
    plt.xlim([0,1])
    plt.title('Distribuição das previsões', size=15)
    plt.xlabel('Probabilidade positiva (previsto)', size=13)
    plt.ylabel('Samples(escala normalizada)', size=13)
    plt.legend(loc="upper right")

    #3 - Curva ROC
    fp_rates, tp_rates, _ = roc_curve(y_test, p[:,1])
    roc_auc = auc(fp_rates, tp_rates)
    plt.subplot(133)
    plt.plot(fp_rates, tp_rates, color = 'purple',
             lw=1, label='Roc curve (area = %0.2f)'%roc_auc)
    plt.plot([0,1], [0,1], lw=1, linestyle='--', color='red')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('Falso positivo', size=13)
    plt.ylabel('Verdadeiro positivo', size=13)
    plt.title('Curva ROC', size=15)
    plt.legend(loc='lower right')
    plt.subplots_adjust(wspace=.3)
    plt.show()
    
    # Mostrar e retornar o f1 score
    tn, fp, fn, tp = [i for i in cm_rl.ravel()]
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    F1 = 2*(precision*recall)/(precision+recall)
    printout=(
        f'Precision: {round(precision,2)} | '
        f'Recall: {round(recall,2)} | '
        f'F1 Score: {round(F1,2)} | '
    )
    print(printout)
    return F1


In [ ]:
F1= evalBinaryClassifierRL(votos_logistic, X_test, y_test)